本文主要内容来自：
- https://zhuanlan.zhihu.com/p/20767428
- https://www.zhihu.com/question/25097993?from=profile_question_card
- https://www.zhihu.com/question/59201590
- https://www.zhihu.com/question/24529483

### train loss与test loss结果分析

train loss 不断下降，test loss不断下降，说明网络仍在学习;<br>
train loss 不断下降，test loss趋于不变，说明网络过拟合;<br>
train loss 趋于不变，test loss不断下降，说明数据集100%有问题;<br>
train loss 趋于不变，test loss趋于不变，说明学习遇到瓶颈，需要减小学习率或批量数目;<br>
train loss 不断上升，test loss不断上升，说明网络结构设计不当，训练超参数设置不当，数据集经过清洗等问题。<br>

### 学习率衰减

下面的代码示范了如何每30个epoch按10%的速率衰减

In [ ]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = args.lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

其中optimizer通过param_group来管理参数组.param_group中保存了参数组及其对应的学习率,动量等等.所以我们可以通过更改param_group[‘lr’]的值来更改对应参数组的学习率

也可以使用，pytorch提供的一些方法来调整学习率：
- torch.optim.lr_scheduler 提供了几种方法来根据epoches的数量调整学习率。 
- torch.optim.lr_scheduler.ReduceLROnPlateau允许基于一些验证测量来降低动态学习速率。

##### class torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda, last_epoch=-1)
将每个参数组的学习速率设置为初始的lr乘以一个给定的函数。当last_epoch=-1时，将初始lr设置为lr。
- optimizer (Optimizer) – 包装的优化器。
- lr_lambda (function or list) – 一个函数来计算一个乘法因子给定一个整数参数的epoch，或列表等功能，为每个组optimizer.param_groups。
- last_epoch (int) – 最后一个时期的索引。默认: -1.

```
lambda1 = lambda epoch: 0.1 ** (epoch // 30)
lambda2 = lambda epoch: 0.95 ** epoch
scheduler = LambdaLR(optimizer, lr_lambda=[lambda1, lambda2])
for epoch in range(100):
    scheduler.step()
    train(...)
    validate(...)
```

##### class torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=False, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
当指标停止改善时，降低学习率。当学习停滞不前时，模型往往会使学习速度降低2-10倍。这个调度程序读取一个指标量，如果没有提高epochs的数量，学习率就会降低。
- optimizer (Optimizer) – 包装的优化器。
- mode (str) – min, max中的一个. 在最小模式下，当监测量停止下降时，lr将减少; 在最大模式下，当监控量停止增加时，会减少。默认值：'min'。
- factor (float) – 使学习率降低的因素。 new_lr = lr * factor. 默认: 0.1.
- patience (int) –epochs没有改善后，学习率将降低。 默认: 10.
- verbose (bool) – 如果为True，则会向每个更新的stdout打印一条消息。 默认: False.
- threshold (float) – 测量新的最优值的阈值，只关注显着变化。 默认: 1e-4.
- threshold_mode (str) – rel, abs中的一个. 在rel模型, dynamic_threshold = best ( 1 + threshold ) in ‘max’ mode or best ( 1 - threshold ) 在最小模型. 在绝对值模型中, dynamic_threshold = best + threshold 在最大模式或最佳阈值最小模式. 默认: ‘rel’.
- cooldown (int) – 在lr减少后恢复正常运行之前等待的时期数。默认的: 0.
- min_lr (float or list) – 标量或标量的列表。对所有的组群或每组的学习速率的一个较低的限制。 默认: 0.
- eps (float) – 适用于lr的最小衰减。如果新旧lr之间的差异小于eps，则更新将被忽略。默认: 1e-8.

```
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
scheduler = torch.optim.ReduceLROnPlateau(optimizer, 'min')
for epoch in range(10):
    train(...)
    val_loss = validate(...)
    # Note that step should be called after validate()
    scheduler.step(val_loss)
```

### 参数初始化

下面几种方式,随便选一个,结果基本都差不多。但是一定要做。否则可能会减慢收敛速度，影响收敛结果，甚至造成Nan等一系列问题。<br>
n_in为网络的输入大小，n_out为网络的输出大小，n为n_in或(n_in+n_out)*0.5<br>
- Xavier初始法论文：http://jmlr.org/proceedings/papers/v9/glorot10a/glorot10a.pdf
- He初始化论文：https://arxiv.org/abs/1502.01852

##### uniform均匀分布初始化：
```
w = np.random.uniform(low=-scale, high=scale, size=[n_in,n_out])
```
- Xavier初始法，适用于普通激活函数(tanh,sigmoid)：scale = np.sqrt(3/n)
- He初始化，适用于ReLU：scale = np.sqrt(6/n)

###### normal高斯分布初始化：
```
w = np.random.randn(n_in,n_out) * stdev # stdev为高斯分布的标准差，均值设为0
```
- Xavier初始法，适用于普通激活函数 (tanh,sigmoid)：stdev = np.sqrt(n)
- He初始化，适用于ReLU：stdev = np.sqrt(2/n)
- svd初始化：对RNN有比较好的效果。参考论文：https://arxiv.org/abs/1312.6120

### 数据预处理

zero-center ,这个挺常用的.
- X -= np.mean(X, axis = 0) # zero-center
- X /= np.std(X, axis = 0) # normalize

PCA whitening,这个用的比较少
```
class sklearn.decomposition.PCA(n_components=None, copy=True, whiten=False, svd_solver=’auto’, tol=0.0, iterated_power=’auto’, random_state=None)

from sklearn.decomposition import PCA
pca=PCA()
pca.fit(X)
pca.transform(X)
```

### 训练

##### 基本原则
快速试错
##### 一些大的注意事项
1. 刚开始, 先上小规模数据, 模型往大了放, 只要不爆显存, 能用256个filter你就别用128个. 直接奔着过拟合去. 没错, 就是训练过拟合网络, 连测试集验证集这些都可以不用.
    - 你要验证自己的训练脚本的流程对不对. 这一步小数据量, 生成速度快, 但是所有的脚本都是和未来大规模训练一致的(除了少跑点循环)
    - 如果小数据量下, 你这么粗暴的大网络奔着过拟合去都没效果. 那么, 你要开始反思自己了, 模型的输入输出是不是有问题? 要不要检查自己的代码(永远不要怀疑工具库, 除非你动过代码)? 模型解决的问题定义是不是有问题? 你对应用场景的理解是不是有错? 不要怀疑NN的能力, 不要怀疑NN的能力, 不要怀疑NN的能力. 就我们调参狗能遇到的问题, NN没法拟合的, 这概率是有多小?

2. Loss设计要合理
    - 一般来说分类就是Softmax, 回归就是L2的loss. 但是要注意loss的错误范围(主要是回归), 你预测一个label是10000的值, 模型输出0, 你算算这loss多大, 这还是单变量的情况下. 一般结果都是nan. 所以不仅仅输入要做normalization, 输出也要这么弄
    - 多任务情况下, 各loss想法限制在一个量级上, 或者最终限制在一个量级上, 初期可以着重一个任务的loss
3. 观察loss胜于观察准确率
    - 准确率虽然是评测指标, 但是训练过程中还是要注意loss的. 你会发现有些情况下, 准确率是突变的, 原来一直是0, 可能保持上千迭代, 然后突然变1. 要是因为这个你提前中断训练了, 只有老天替你惋惜了. 而loss是不会有这么诡异的情况发生的, 毕竟优化目标是loss
    - 给NN一点时间, 要根据任务留给NN的学习一定空间. 不能说前面一段时间没起色就不管了. 有些情况下就是前面一段时间看不出起色, 然后开始稳定学习
4. 确认分类网络充分学习
5. 用momentum来加快收敛
    - 对于一般的SGD，其表达式为 x = x - a * dx,沿负梯度方向下降。而带momentum项的SGD则写生如下形式：v = b*v - a*dx; x = x + v 其中b即momentum系数，通俗的理解上面式子就是，如果上一次的momentum（即）与这一次的负梯度方向是相同的，那这次下降的幅度就会加大，所以这样做能够达到加速收敛的过程
5. Learning Rate设置合理
    - 太大: loss爆炸, 或者nan
    - 太小: 半天loss没反映
    - 需要进一步降低了: loss在当前LR下一路降了下来, 但是半天不再降了
    - 如果有个复杂点的任务, 刚开始, 是需要人肉盯着调LR的. 后面熟悉这个任务网络学习的特性后, 可以扔一边跑去了
    - 如果上面的Loss设计那块你没法合理, 初始情况下容易爆, 先上一个小LR保证不爆, 等loss降下来了, 再慢慢升LR, 之后当然还会慢慢再降LR, 虽然这很蛋疼
6. 对比训练集和验证集的loss
    - 判断过拟合, 训练是否足够, 是否需要early stop的依据
7. 清楚receptive field的大小??
    - CV的任务, context window是很重要的. 所以你对自己模型的receptive field的大小要心中有数. 这个对效果的影响还是很显著的. 特别是用FCN, 大目标需要很大的receptive field. 不像有fully connection的网络, 好歹有个fc兜底, 全局信息都有

### 经验

- 数据不足的时候，做data argument
- shuffle, shuffle, shuffle
- dropout对小数据防止过拟合有很好的效果,值一般设为0.5,小数据上dropout+sgd在我的大部分实验中，效果提升都非常明显.因此可能的话，建议一定要尝试一下。 dropout的位置比较有讲究, 对于RNN,建议放到输入->RNN与RNN->输出的位置.
- adam,adadelta等,在小数据上,我这里实验的效果不如sgd, sgd收敛速度会慢一些，但是最终收敛后的结果，一般都比较好。如果使用sgd的话,可以选择从1.0或者0.1的学习率开始,隔一段时间,在验证集上检查一下,如果cost没有下降,就对学习率减半. 我看过很多论文都这么搞,我自己实验的结果也很好. 
- 也可以先用ada系列先跑,最后快收敛的时候,更换成sgd继续训练.同样也会有提升.据说adadelta一般在分类问题上效果比较好，adam在生成问题上效果比较好。
- rnn的dim和embdding size,一般从128上下开始调整. batch size,一般从128左右开始调整.batch size合适最重要,并不是越大越好
- LSTM 的forget gate的bias,用1.0或者更大的值做初始化,可以取得更好的结果
- 如果你的模型包含全连接层（MLP），并且输入和输出大小一样，可以考虑将MLP替换成Highway Network
- CNN更加适合训练回答是否的问题, 如果任务比较复杂, 考虑先用分类任务训练一个模型再finetune.
- 无脑用ReLU(CV领域)
- 无脑用3x3
- 无脑用xavier
- filter数量2^n
- 多尺度的图片输入(或者网络内部利用多尺度下的结果)有很好的提升效果
- 第一层的filter, 数量不要太少. 否则根本学不出来(底层特征很重要)
- sgd adam 这些选择上, 看你个人选择. 一般对网络不是决定性的. 反正我无脑用sgd + momentum
- shortcut的联接是有作用的
- 暴力调参最可取, 毕竟, 自己的生命最重要. 你调完这个模型说不定过两天这模型就扔掉了

### Overfitting

##### 获得更多数据
- 从数据源头获得更多数据
- 根据当前数据集，估计数据分布，使用该分布产生更多的数据
- 数据增强（data argument）：通过一定的规则扩充数据。如物体分类里面，物体图中的位置、姿态、尺度，整体图像明暗度等都不会影响分类结果，我们可以通过这些方法来使得数据库成倍扩增

##### 使用合适的模型
- 减少网络层数，神经元个数等都可以限制网络的拟合能力
- early stop
- weight-decay为正则项前面的系数，提高weight decay可以缓解high variance的问题（即Overfitting）
- 增加噪声
    - 输入中加噪声
    ```
    means=torch.zeros(s1,s2,s3)
    std = torch.ones(s1,s2,s3)
    sigma = 15
    std = std*sigma
    noise = torch.normal(means, std)
    ```
    - 权值上加噪声：在初始化网络的时候，用0均值的高斯分布作为初始化
    - 对网络的响应加噪声：如在前向传播的过程中，让某些神经元的输出变为binary或random。显然这种有些乱来的做法会打乱网络的训练过程，让训练变慢，但据Hinton说，在测试集上效果会显著提升
- 使用多种模型: 训练多个模型，以每个模型的平均输出作为结果
    - Bagging 简单理解，就是分段函数的概念，用不同的模型拟合不同部分的训练集。以随机森林为例，就是训练一堆互不关联的决策树。但由于训练神经网络本身就需要耗费比较多自由，所以一般不单独使用神经网络做bagging
    - Boosting 通过训练一系列的简单神经网络，加权平均其输出
    - Dropout
    - 贝叶斯方法???

### Ensemble
Ensemble是论文刷结果的终极核武器,深度学习中一般有以下几种方式
- 同样的参数,不同的初始化方式
- 不同的参数,通过cross-validation,选取最好的几组
- 同样的参数,模型训练的不同阶段，即不同迭代次数的模型。
- 不同的模型,进行线性融合. 例如RNN和传统模型.